In [1]:

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import shutil
from collections import OrderedDict
import subprocess
import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch
torch.set_num_threads(1)

import nnunetv2

import h5py


In [2]:
def make_if_dont_exist(folder_path, overwrite=False):
    """
    creates a folder if it does not exists
    input:
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder
    """
    if os.path.exists(folder_path):

        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
        os.makedirs(folder_path)
        print(f"{folder_path} created!")

In [8]:
dataset = "Dataset001_Kaggle"

In [4]:
print("Current Working Directory {}".format(os.getcwd()))

Current Working Directory C:\Users\paula\PycharmProjects\STT_classification\Segmentation\nnUNetFrame


In [6]:
nnUNet_raw = os.path.join('C:\\Users\\paula\\PycharmProjects\\STT_classification\\Segmentation\\nnUNetFrame', 'nnUNet_raw')


In [9]:
# Create directories for train and test data
train_image_dir = os.path.join(nnUNet_raw, dataset, 'imagesTr')
train_label_dir = os.path.join(nnUNet_raw, dataset, 'labelsTr')
test_image_dir = os.path.join(nnUNet_raw, dataset, 'imagesTs')
test_label_dir = os.path.join(nnUNet_raw, dataset, 'labelsTs')

make_if_dont_exist(train_image_dir, overwrite=True)
make_if_dont_exist(train_label_dir, overwrite=True)
make_if_dont_exist(test_image_dir, overwrite=True)
make_if_dont_exist(test_label_dir, overwrite=True)

C:\Users\paula\PycharmProjects\STT_classification\Segmentation\nnUNetFrame\nnUNet_raw\Dataset001_Kaggle\imagesTr overwritten
C:\Users\paula\PycharmProjects\STT_classification\Segmentation\nnUNetFrame\nnUNet_raw\Dataset001_Kaggle\labelsTr overwritten
C:\Users\paula\PycharmProjects\STT_classification\Segmentation\nnUNetFrame\nnUNet_raw\Dataset001_Kaggle\imagesTs overwritten
C:\Users\paula\PycharmProjects\STT_classification\Segmentation\nnUNetFrame\nnUNet_raw\Dataset001_Kaggle\labelsTs overwritten


In [ ]:
source_images = os.path.join("RawData", 'imagesTr')
source_labels = os.path.join("RawData", 'labelsTr')

In [ ]:
# Iterate over the MRI images and masks

for file in os.listdir(source_images):
    if file.endswith('.nii.gz'):
        patient_id = file.split('.')[0]  # Extract patient ID (assuming file name is just the ID)

        # Rename and move the image
        new_image_name = f"{patient_id}_0000.nii.gz"
        shutil.move(os.path.join(source_images, file), os.path.join(train_image_dir, new_image_name))

        # Rename and move the mask (if it exists)
        mask_file = os.path.join(source_labels, f"{patient_id}.nii.gz")
        if os.path.exists(mask_file):
            shutil.move(mask_file, os.path.join(train_label_dir, f"{patient_id}.nii.gz"))

print("Renaming and moving complete!")

In [ ]:
def create_json(train_image_dir, test_image_dir):
    overwrite_json_file = True  # make it True if you want to overwrite the dataset.json file in Dataset_folder
    json_file_exist = False

    if os.path.exists(os.path.join(nnUNet_raw, 'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True

    if json_file_exist == False or overwrite_json_file:

        json_dict = OrderedDict()
        json_dict['name'] = "Soft_tissue_tumours Kaggle"
        json_dict['description'] = "Soft Tissue Segmentation Kaggle"
        json_dict['tensorImageSize'] = "3D"
        #json_dict['reference'] = "see challenge website"
        #json_dict['licence'] = "see challenge website"
        #json_dict['release'] = "0.0"

        # you may mention more than one modality
        json_dict['channel_names'] = {
            "0": "MRI"
        }

        # set expected file ending
        json_dict["file_ending"] = ".nii.gz"

        # label names should be mentioned for all the labels in the dataset
        json_dict['labels'] = {
            "background": 0,
            "tumour": 1
        }

        train_ids = os.listdir(train_image_dir)
        test_ids = os.listdir(test_image_dir)
        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        with open(os.path.join(nnUNet_raw, "dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=True)

        if os.path.exists(os.path.join(nnUNet_raw, 'dataset.json')):
            if json_file_exist == False:
                print('dataset.json created!')
            else:
                print('dataset.json overwritten!')
